# Welcome to this notebook! 

Here, we will be working with the KBO database to perform various SQL queries and analyses. The notebook contains several sections where we will execute SQL commands, manipulate data, and visualize results.

Outside of the notebook, we will also be exporting results to CSV files, allowing us to present some charts on powerpoint slides. You will find the slides in the folder.

We used others software to read and query the database : table plus, DBeaver and finally installed Database Client as an extension in Visual Studio Code. We used DBeaver a lot to have the schema of the database and to understand how to query it.



In [ ]:
# We are using this notebook to make the SQL queries from the KBO database. 
import sqlite3
import pandas as pd

db_path = 'kbo_database.db'

# Connect to SQLite database (creates the file if it does not exist)
connect = sqlite3.connect(db_path)

# Create a cursor object
cursor = connect.cursor()


## 1. Understanding the Database

In [38]:
"""
Query to get the distribution of companies by juridical form with percentages. First, we used the same technique as in the exercices of 
Bouman Github. Getting the query in a variable and running it with the cursor.connect 

This query is about the distribution of companies by juridical form, showing the top 10 forms with their counts and percentages."""

querypourcentage = """
WITH grouped AS (
SELECT enterprise.JuridicalForm , 
code.Description, 
COUNT(*) AS counts
FROM enterprise
JOIN code ON enterprise.JuridicalForm = code.Code
AND code."Language" = 'FR'
WHERE code.Category = 'JuridicalForm'
GROUP BY enterprise.JuridicalForm
ORDER BY COUNT(*) DESC), 
total AS (
SELECT SUM(counts) AS total_count
FROM grouped)
SELECT grouped.JuridicalForm, 
grouped.Description, 
grouped.counts, ROUND(grouped.counts * 100.0 / total.total_count, 2) AS Pourcentage
FROM grouped, total
ORDER BY Pourcentage DESC
LIMIT 10;
"""
cursor.execute(querypourcentage)
resultpourcentage = cursor.fetchall()
for JuridicalFrom, Description, counts, Pourcentage in resultpourcentage:
    print(f"{JuridicalFrom} - {Description}: {counts} ({Pourcentage}%)")    
# We used a loop to get a nice output

610.0 - Société à responsabilité limitée: 493384 (42.78%)
17.0 - Association sans but lucratif: 139213 (12.07%)
70.0 - Association des copropriétaires: 108445 (9.4%)
15.0 - Société privée à responsabilité limitée: 86541 (7.5%)
14.0 - Société anonyme: 80675 (7.0%)
30.0 - Entité étrangère: 67576 (5.86%)
612.0 - Société en commandite: 47352 (4.11%)
721.0 - Société ou association sans personnalité juridique: 27737 (2.41%)
11.0 - Société en nom collectif: 24888 (2.16%)
12.0 - Société en commandite simple: 16182 (1.4%)


In [39]:
# Query to get the distribution of companies status. Not sure about this one. 

querystatus = """
SELECT Status, COUNT(*) AS Counts  
FROM enterprise
GROUP BY Status
LIMIT 10;
"""
cursor.execute(querystatus)
resultstatus = cursor.fetchall()
for Status, Counts in resultstatus:
    print(f"{Status}: {Counts} companies")

AC: 1926246 companies


In [40]:
# Query to get the activity group types in French and see if it is relevant. But we won't use that later.
queryactivitygrouptype = """SELECT *
FROM code
WHERE "Language" = 'FR' AND "Category" = "ActivityGroup";
"""
cursor.execute(queryactivitygrouptype)
result = cursor.fetchall() 
for row in result:
    print(row)

('ActivityGroup', '001', 'FR', 'Activités TVA')
('ActivityGroup', '002', 'FR', 'Activités EDRL')
('ActivityGroup', '003', 'FR', 'Activités')
('ActivityGroup', '004', 'FR', 'Activités de la fonction publique fédérale')
('ActivityGroup', '005', 'FR', 'Activités ONSSAPL')
('ActivityGroup', '006', 'FR', 'Activités ONSS')
('ActivityGroup', '007', 'FR', "Activités de l'enseignement subventionné")


In [41]:
# Query to get the activity group types by startdate but only where activity group = 001 and 002
queryactivitygroup = """
SELECT enterprise.date_creation_dt, code.Description
FROM enterprise
JOIN activity ON activity.EntityNumber = enterprise.EnterpriseNumber
JOIN code ON activity.ActivityGroup = code.Code
WHERE code.Language = 'FR' 
  AND code.Category = 'ActivityGroup'
  AND (code.Code = '001' OR code.Code = '002')
LIMIT 10;
"""
cursor.execute(queryactivitygroup)
resultactivitygroup = cursor.fetchall()
for date_creation_dt, Description in resultactivitygroup:
    print(f"{date_creation_dt} - {Description}")

    

1960-08-09 - Activités TVA
1960-08-09 - Activités TVA
1960-08-09 - Activités TVA
1923-02-16 - Activités TVA
1923-02-16 - Activités TVA
1923-02-16 - Activités TVA
1923-02-16 - Activités TVA
1923-02-16 - Activités TVA
1923-02-16 - Activités TVA
1923-02-16 - Activités TVA


## 2. Cleaning the Data for further Analysis

At this point, we have tested some queries and are ready to proceed with the next steps. We decided to implement the enterprise table in the database with the different columns we need from other tables. Allowing us to have a more complete table with the information we need for our analysis.

In [ ]:
# Add Zipcode column to enterprise table if it does not exist. We would like to use zipcode to analyze by province or city. 
try:
    cursor.execute("ALTER TABLE enterprise ADD COLUMN Zipcode TEXT")
except sqlite3.OperationalError:
    # Column already exists
    pass

queryaddzipcode = """
UPDATE enterprise
SET Zipcode = (
    SELECT address.Zipcode
    FROM address
    WHERE address.EntityNumber = enterprise.EnterpriseNumber
)
"""
cursor.execute(queryaddzipcode)
connect.commit()


In [43]:
#Adding Province column to enterprise table if it does not exist
try:
    cursor.execute("ALTER TABLE enterprise ADD COLUMN Province TEXT")
except sqlite3.OperationalError:
    # Column already exists
    pass    
queryaddprovince = """
UPDATE enterprise
SET Province = 
CASE
    WHEN ZipCode LIKE '1%' THEN 'Bruxelles'
    WHEN ZipCode LIKE '2%' THEN 'Brabant flamand'
    WHEN ZipCode LIKE '3%' THEN 'Liège'
    WHEN ZipCode LIKE '4%' THEN 'Liège'
    WHEN ZipCode LIKE '5%' THEN 'Namur'
    WHEN ZipCode LIKE '6%' THEN 'Hainaut'
    WHEN ZipCode LIKE '7%' THEN 'Limbourg'
    WHEN ZipCode LIKE '8%' THEN 'Flandre orientale'
    WHEN ZipCode LIKE '9%' THEN 'Flandre occidentale'
    ELSE 'Inconnu'
  END;
"""

cursor.execute(queryaddprovince)
connect.commit()

At this point, we have seen that the zipcode didn't correspond to reality but we wen't far behind (we already had several charts in slides) but Luxembourg didn't appear so we had to update the zipcode column with the correct values. Fortunately, we only had to update the csv and Excel and Powerpoint synch with the new data.

In [ ]:
# We used a really really precise query to update the Province column based on the ZipCode values.
# From this moment, we also put directly the sql query in the connect.execute() method.
connect.execute("""
UPDATE horecabasis
SET Province =
  CASE
    WHEN ZipCode BETWEEN 1000 AND 1299 THEN 'Bruxelles-Capitale'

    WHEN ZipCode BETWEEN 1300 AND 1499 THEN 'Brabant wallon'
    WHEN ZipCode BETWEEN 1500 AND 1999 THEN 'Brabant flamand'
    WHEN ZipCode BETWEEN 3000 AND 3499 THEN 'Brabant flamand'

    WHEN ZipCode BETWEEN 2000 AND 2999 THEN 'Anvers'

    WHEN ZipCode BETWEEN 3500 AND 3999 THEN 'Limbourg'

    WHEN ZipCode BETWEEN 4000 AND 4999 THEN 'Liège'

    WHEN ZipCode BETWEEN 5000 AND 5999 THEN 'Namur'

    WHEN ZipCode BETWEEN 6000 AND 6599 THEN 'Hainaut'
    WHEN ZipCode BETWEEN 7000 AND 7999 THEN 'Hainaut'

    WHEN ZipCode BETWEEN 6600 AND 6999 THEN 'Luxembourg'

    WHEN ZipCode BETWEEN 8000 AND 8999 THEN 'Flandre-Occidentale'

    WHEN ZipCode BETWEEN 9000 AND 9999 THEN 'Flandre-Orientale'

    ELSE 'Inconnu'
  END;
""")
connect.commit()

In [45]:
# Adding column for the year of creation
try:
	cursor.execute("ALTER TABLE enterprise ADD COLUMN YearOfCreation INTEGER")
except sqlite3.OperationalError:
	# Column already exists
	pass

# Then, update the column with the year extracted from the Date column
cursor.execute("""
UPDATE enterprise
SET YearOfCreation = CAST(substr(date_creation_dt, 1, 4) AS INTEGER)
WHERE date_creation_dt IS NOT NULL;
""")
connect.commit()

In [ ]:
# Adding column for the month of creation
try:
	cursor.execute("ALTER TABLE enterprise ADD COLUMN MonthOfCreation INTEGER")
except sqlite3.OperationalError:
	# Column already exists
	pass

# Then, update the column with the month extracted from the Date column
cursor.execute("""
UPDATE enterprise
SET MonthOfCreation = CAST(strftime('%m', date_creation_dt) AS INTEGER)
WHERE date_creation_dt IS NOT NULL;
""")
connect.commit()

In [ ]:
# Add NaceCode column to enterprise table
try:
    cursor.execute("ALTER TABLE enterprise ADD COLUMN Nacecode TEXT")
except sqlite3.OperationalError:
    # Column already exists
    pass

cursor.execute("""
UPDATE enterprise
SET Nacecode = (
    SELECT activity.NaceCode
    FROM activity
    WHERE activity.EntityNumber = enterprise.EnterpriseNumber
    LIMIT 1
)
""")
connect.commit()

In [ ]:
# Now we create a new table who will be the basis for our Horeca analysis. 
cursor.execute("""
CREATE TABLE horecabasis AS
SELECT * FROM enterprise;
""")
connect.commit()

OperationalError: table horecabasis already exists

In [ ]:
# We preferred to do a copy of the table because we now delete all the rows that doesn't matter meaning 
# the rows that are not nacecode 55 and 56 (corresponding to horeca activities).
cursor.execute("""
DELETE FROM horecabasis
WHERE 
  NaceCode IS NULL
  OR TRIM(NaceCode) = ''
  OR substr(NaceCode, 1, 2) NOT IN ('55', '56');
""")
connect.commit()

## 3. Analysis

Now we do our query to get the numbers we are interested in. Then we save each results in a CSV to directly work on them in powerpoint.

In [ ]:
# query to get the number of horeca companies created per year 
numberperyear = pd.read_sql("""
SELECT 
YearOfCreation, 
COUNT(*) AS nb_HORECA_society_created
FROM horecabasis
GROUP BY YearOfCreation
ORDER BY YearOfCreation DESC;
""", connect)

# Export CSV
numberperyear.to_csv("numberperyear.csv", index=False)

In [ ]:
# query to get the number of horeca companies created by municipality. We limit to 10 descending to have the top 10.

byzipcode = pd.read_sql("""
SELECT Zipcode,
  COUNT(*) AS nb_entreprises 
FROM horecabasis
WHERE Zipcode IS NOT NULL
  AND TRIM(Zipcode) != ''
  AND Zipcode != '0'
  AND Zipcode != 'Inconnu'
GROUP BY Zipcode
ORDER BY nb_entreprises DESC
LIMIT 10;
""", connect)
# Export CSV
byzipcode.to_csv("byzipcode.csv", index=False)

In [ ]:
# This query get the number of companies by decade and province after 1990 because before is irrelevant and make to much noise. 

annualprovinces = pd.read_sql("""
SELECT 
  YearOfCreation,
  Province,
  COUNT(*) AS nb_entreprises
FROM horecabasis
WHERE YearOfCreation >= 1990
  AND Province NOT LIKE 'Inconnu'
GROUP BY YearOfCreation, Province
ORDER BY YearOfCreation, Province;
""", connect)
annualprovinces["by10years"] = (annualprovinces["YearOfCreation"] // 10) * 10
by_decade = annualprovinces.groupby(["by10years", "Province"])["nb_entreprises"].sum().reset_index()
# Export CSV    
by_decade.to_csv("by_decade_by_provinces.csv", index=False)

In [ ]:
# This query get the number of companies created by month. 

monthlyevolution = pd.read_sql("""
SELECT 
  MonthOfCreation,
  COUNT(*) AS nb_entreprises
FROM horecabasis
GROUP BY MonthOfCreation
ORDER BY MonthOfCreation;
""", connect)
# Export CSV
monthlyevolution.to_csv("monthlyevolution.csv", index=False)

In [ ]:
# This query get the number of active companies by province.

presenceinprovinces = pd.read_sql("""
SELECT  
  Province,
  COUNT(*) AS nb_entreprises
FROM horecabasis
WHERE Province IS NOT NULL
  AND TRIM(Province) != ''
  AND Province != '0'
  AND Province != 'Inconnu'
GROUP BY Province
ORDER BY nb_entreprises DESC;
""", connect)
# Export CSV
presenceinprovinces.to_csv("presenceinprovinces.csv", index=False)

## 4. Closing Session

In [50]:
# We close the connection to the database after all operations are done

connect.close()